In [1]:
import sympy as sp
from boson_ham_to_matrix import *
from constants import *

## Diagonal Part

In [2]:
totHam = pn**2 + qn**2

In [3]:
totHam

p_{n}**2 + q_{n}**2

In [14]:
N=3
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}

# warning be careful of array indexing...   
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1]
                         }
elif bcType == 'dirichlet':
    boundaryConditions = {qs[0]: 0, qs[N+1]: 0}

In [15]:
s=''
for q in qs:
    s+='{}  '.format(q)
print(s)
qs[0]

q_{-1}  q_{0}  q_{1}  q_{2}  q_{3}  


q_{-1}

In [16]:
ham=0
for i in range(1,N+1):
    ham+=totHam.subs({qn: qs[i], pn: ps[i]}).subs(boundaryConditions)

ham

p_{0}**2 + p_{1}**2 + p_{2}**2 + q_{0}**2 + q_{1}**2 + q_{2}**2

In [19]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
sp.nsimplify(hoHam,tolerance=1e-8)

a^{\dagger}_{0}*a_{0} + a^{\dagger}_{1}*a_{1} + a^{\dagger}_{2}*a_{2} + a_{0}*a^{\dagger}_{0} + a_{1}*a^{\dagger}_{1} + a_{2}*a^{\dagger}_{2}

In [20]:
cutoff=4
hamMat=convert_to_matrix(hoHam,cutoff,N)

TypeError: Cannot convert expression to float

## Full boson part

In [3]:
bosonNI=sp.expand( p[n]**2/(2*aLat) + (aLat/2)*((q[n+1]-q[n-1])/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(q[n])**2 + aLat*V(q[n])*(q[n+1]-q[n-1])/(2*aLat) )

ham=0
totHam=bosonNI+bosonI
totHam

a*V(q[n])**2/2 + V(q[n])*q[n + 1]/2 - V(q[n])*q[n - 1]/2 + p[n]**2/(2*a) + q[n + 1]**2/(8*a) - q[n + 1]*q[n - 1]/(4*a) + q[n - 1]**2/(8*a)

In [4]:
N=3
aVal=1

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {q[-1]: q[N-1], q[N]: q[0],
                          x[N]: x[0], xd[N]: xd[0],
                         }
elif bcType == 'dirichlet':
    boundaryConditions = {q[-1]: 0, q[N]: 0,
                          x[N]: 0, xd[N]: 0
                         }

In [5]:

for i in range(0,N):
    ham+=totHam.subs(n,i).subs(boundaryConditions)
# ham.subs(boundaryConditions).doit() # this doesn't work?

def potential(n):
    return 0

for n in range(0,N):
    ham=ham.subs(V(q[n]),potential(n))
ham=ham.subs(aLat,aVal).simplify().expand()
ham

p[0]**2/2 + p[1]**2/2 + p[2]**2/2 + q[0]**2/4 - q[0]*q[1]/4 - q[0]*q[2]/4 + q[1]**2/4 - q[1]*q[2]/4 + q[2]**2/4

In [6]:
m=1

HOdofSubs = {}
for i in range(0,N):
    HOdofSubs[q[i]] = 0.5*sp.sqrt(2/m)*(a[i] + adag[i])
    HOdofSubs[p[i]] = complex(0,1)*sp.sqrt(2*m)*(adag[i] - a[i])/2
    
hoHam=sp.expand(ham.subs(HOdofSubs))
sp.nsimplify(hoHam,tolerance=1e-10)

-a[0]**2/8 - a[0]*a[1]/8 - a[0]*a[2]/8 + 3*a[0]*a^{\dagger}[0]/4 - a[0]*a^{\dagger}[1]/8 - a[0]*a^{\dagger}[2]/8 - a[1]**2/8 - a[1]*a[2]/8 - a[1]*a^{\dagger}[0]/8 + 3*a[1]*a^{\dagger}[1]/4 - a[1]*a^{\dagger}[2]/8 - a[2]**2/8 - a[2]*a^{\dagger}[0]/8 - a[2]*a^{\dagger}[1]/8 + 3*a[2]*a^{\dagger}[2]/4 - a^{\dagger}[0]**2/8 - a^{\dagger}[0]*a^{\dagger}[1]/8 - a^{\dagger}[0]*a^{\dagger}[2]/8 - a^{\dagger}[1]**2/8 - a^{\dagger}[1]*a^{\dagger}[2]/8 - a^{\dagger}[2]**2/8

In [7]:
cutoff=4
hamMat=convert_to_matrix(hoHam,cutoff,N)

In [8]:
hamMat.shape

(64, 64)

In [9]:
np.isreal(hamMat).all()

True

In [10]:
(hamMat==np.matrix(hamMat).H).all()

False

In [11]:
np.sort(np.linalg.eig(hamMat.astype(np.complex64))[0])

array([-4.9287577 +1.4904457e-16j, -2.994568  -1.4490519e-15j,
       -2.994568  +6.6613381e-16j, -2.95752   +9.6117847e-16j,
       -1.5382209 -1.8949052e-15j, -1.5382209 -3.6905564e-16j,
       -1.527777  -1.7678757e-16j, -1.3447806 -6.2201033e-16j,
       -1.3447806 +3.8478134e-16j, -1.2956979 +1.9968863e-17j,
       -0.67994785-3.8959111e-16j, -0.67994785-1.8731571e-16j,
       -0.6731258 -4.7969795e-17j, -0.39187726-2.9924658e-16j,
       -0.06570498-7.4490696e-01j, -0.06570498+7.4490696e-01j,
       -0.0104075 -1.4855193e-01j, -0.0104075 -1.4855193e-01j,
       -0.0104075 +1.4855193e-01j, -0.0104075 +1.4855193e-01j,
        0.04744779-3.7223721e-01j,  0.04744779+3.7223721e-01j,
        0.125     +8.0478050e-17j,  0.13794218-1.5831737e-01j,
        0.13794218-1.5831737e-01j,  0.13794218+1.5831737e-01j,
        0.13794218+1.5831737e-01j,  0.30478805-8.3254881e-02j,
        0.30478805-8.3254881e-02j,  0.30478805+8.3254881e-02j,
        0.30478805+8.3254881e-02j,  0.3280198 +6.874618

In [12]:
m=convert_term_to_matrix(1.0*a[0]*a[1], 4, 3).astype(float)

In [13]:
for row in m:
    print(row)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.   